In [1]:
import pandas as pd
from pathlib import Path

import geopandas as gpd
from shapely.geometry import LineString
import contextily as ctx
import matplotlib.pyplot as plt

import numpy as np
from linearmodels import PanelOLS

BAYWHEELS = Path(r"C:\Users\chris\OneDrive\Desktop\baywheels_sf.csv")
RIDERSHIP = Path(r"C:\Users\chris\Downloads\RidershipbyRouteTableDownload.csv")

GTFS_ROUTES = Path(r"C:\Users\chris\OneDrive\Desktop\routes.txt")
GTFS_TRIPS = Path(r"C:\Users\chris\OneDrive\Desktop\trips.txt")
GTFS_STOPS = Path(r"C:\Users\chris\OneDrive\Desktop\stops.txt")

GTFS_STOP_TIMES = Path(r"C:\Users\chris\OneDrive\Desktop\stop_times.txt")
GTFS_SHAPES = Path(r"C:\Users\chris\OneDrive\Desktop\shapes.txt")

In [2]:
baywheels_sf = pd.read_csv(BAYWHEELS, engine = 'pyarrow')         # REMEMBER: A STATION ID CAN HAVE MULTIPLE STATION NAMES BECAUSE THESE STATIONS ARE VERY CLOSE TOGETHER SO I ASSIGNED THEM ALL THE SAME STATION ID  
muni = pd.read_csv(RIDERSHIP, engine = 'pyarrow')

routes = pd.read_csv(GTFS_ROUTES, engine = 'pyarrow')
trips = pd.read_csv(GTFS_TRIPS, engine = 'pyarrow')
stops = pd.read_csv(GTFS_STOPS, engine = 'pyarrow')
stop_times = pd.read_csv(GTFS_STOP_TIMES, engine = 'pyarrow')
shapes = pd.read_csv(GTFS_SHAPES, engine = 'pyarrow')

In [3]:
# IF A STATION ID HAS MULTIPLE STATION NAMES IT SHOULDN'T MATTER WHICH STATION NAME'S COORDINATES IS CHOSEN BECAUSE I SET THEM TO BE ALL THE SAME 
baywheels_sf = baywheels_sf[baywheels_sf['end_station_id'] != 'SF-Y7'].copy() 
baywheels_sf = baywheels_sf[baywheels_sf['started_at'] > '2019-06-01'].copy()

bikeshare_stations = baywheels_sf.sort_values('ended_at').drop_duplicates(subset = ['end_station_id'], keep = 'first').copy()
bikeshare_stations = bikeshare_stations[['end_station_id', 'end_lat', 'end_lng', 'ended_at']].rename(columns = {'ended_at': 'first_appeared_at'})
bikeshare_stations['first_appeared_at'] = pd.to_datetime(bikeshare_stations['first_appeared_at']).dt.normalize()

In [4]:
muni['Month'] = pd.to_datetime(muni['Month'], format = '%B %Y').dt.normalize()

muni.dropna(subset = ['Average Daily Boardings'], inplace = True)
muni['Average Daily Boardings'] = muni['Average Daily Boardings'].str.replace(',', '').astype('int64')

BUS_SERIVCE_CATEGORIES = ['Frequent Local', 'Grid', 'Rapid Bus', 'Connector']
muni = muni[muni['Service Category'].isin(BUS_SERIVCE_CATEGORIES)].copy()
muni['Route'] = muni['Route'].str.upper()
muni = muni[muni['Service Day of the Week'] == 'Weekday'][['Month', 'Route', 'Average Daily Boardings']].copy()

In [5]:
bus_routes = routes[routes['route_type'] == 3].copy()
bus_trips = trips[trips['route_id'].isin(bus_routes['route_id'])].copy()
bus_stop_times = stop_times[stop_times['trip_id'].isin(bus_trips['trip_id'])].copy()

bus_stop_times = bus_stop_times.merge(stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on = 'stop_id', how = 'left')
bus_trips = bus_trips.merge(bus_routes[['route_id', 'route_short_name', 'route_long_name']], on = 'route_id', how = 'left')

bus_route_stops = bus_stop_times.merge(bus_trips[['trip_id', 'route_id', 'route_short_name', 'route_long_name', 'direction_id', 'trip_headsign']])

route_stops = bus_route_stops.sort_values(['route_id', 'direction_id', 'stop_sequence']).drop_duplicates(['route_id', 'direction_id', 'stop_id'])

keep_route_stop_columns = ['stop_id', 'direction_id', 'stop_sequence', 'stop_name', 'route_short_name', 'route_long_name', 'stop_lat', 'stop_lon']
route_stops = route_stops[keep_route_stop_columns]

route_stops = route_stops[route_stops['direction_id'] == 1].copy()              # PICK A DIRECTION: I PICKED INBOUND

In [6]:
shapes = shapes.sort_values(['shape_id', 'shape_pt_sequence'])

bus_shapes = shapes.groupby('shape_id')[['shape_pt_lon', 'shape_pt_lat']].apply(lambda df: LineString(zip(df['shape_pt_lon'], df['shape_pt_lat']))).reset_index(name = 'geometry')
bus_shapes_gdf = gpd.GeoDataFrame(bus_shapes, geometry = 'geometry', crs = 'EPSG:4326')

bus_shape_routes = bus_shapes_gdf.merge(bus_trips.drop_duplicates('shape_id'), on = 'shape_id', how = 'left')
bus_shape_routes = bus_shape_routes[bus_shape_routes['direction_id'] == 1.0]
bus_shape_routes = bus_shape_routes[bus_shape_routes['route_id'].notna()].copy()

In [14]:
import matplotlib.animation as animation

bus_df = bus_shape_routes
bike_df = bikeshare_stations

monthly_dates = pd.date_range(start = bike_df['first_appeared_at'].min(), end = bike_df['first_appeared_at'].max(), freq = 'ME')

bus_gdf = gpd.GeoDataFrame(bus_df, geometry = 'geometry', crs = 'EPSG: 4326')

bus_gdf_3857 = bus_gdf.to_crs(epsg = 3857)

bike_gdf = gpd.GeoDataFrame(bike_df, geometry = gpd.points_from_xy(bike_df.end_lng, bike_df.end_lat), crs = 'EPSG:4326')

bike_gdf_3857 = bike_gdf.to_crs(epsg = 3857)

bike_gdf_3857['x_3857'] = bike_gdf_3857.geometry.x
bike_gdf_3857['y_3857'] = bike_gdf_3857.geometry.y

fig, ax = plt.subplots(figsize = (15,15))
ax.axis('off')
fig.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1)

bus_gdf_3857.plot(ax = ax, color = 'blue', alpha = 0.2, linewidth = 1)
ctx.add_basemap(ax, source = ctx.providers.CartoDB.Positron)

scat = ax.scatter([], [], c = 'red', s = 30, alpha = 0.9, edgecolors = 'white', linewidth = 0.5, zorder = 5)
date_text = ax.text(0.02, 0.95, '', transform = ax.transAxes, fontsize = 12, bbox = dict(facecolor = 'white', alpha = 0.9, boxstyle = 'round'))

def update(frame_date):
    current_data = bike_gdf_3857[bike_gdf_3857['first_appeared_at'] <= frame_date]

    if not current_data.empty:
        scat.set_offsets(np.c_[current_data['x_3857'], current_data['y_3857']])
    
    date_text.set_text(frame_date.strftime('%B %Y'))
    return scat, date_text

ani = animation.FuncAnimation(fig, update, frames = monthly_dates, interval = 100, blit = True)
output_file = 'sf_bikeshare_growth.gif'
ani.save(output_file, writer = 'pillow', fps = 2, dpi = 150)
plt.close()

In [8]:
muni_route_number_dictionary = {
    '21': '6'
}

muni_route_long_name_dictionary = {
    'HAYES': 'HAYES/PARNASSUS',
    'HAIGHT/PARNASSUS': 'HAYES/PARNASSUS'
}

muni_drop_suspended_routes_list = ['JACKSON', 'TOWNSEND', 'VAN NESS']       # SUSPENDED

standardize_route_stops_long_name_dictionary = {
    'HAYES-PARNASSUS': 'HAYES/PARNASSUS',       # THE 6 HAYES-PERNASSUS REPLACED 21 HAYES AND 6 HAIGHT/PARNASSUS
    'HAIGHT-NORIEGA': 'HAIGHT/NORIEGA',
    'FOLSOM-PACIFIC': 'FOLSOM/PACIFIC',
    'ASHBURY-18TH ST': 'ASHBURY/18TH',
    'UNION-STOCKTON': 'UNION/STOCKTON',
    'VAN NESS-MISSION': 'VAN NESS/MISSION',
    'QUINTARA-24TH STREET': 'QUINTARA/24TH STREET',

}

route_stops_drop_routes_list = ['BAYVIEW HUNTERS POINT EXPRESS','CALIFORNIA EXPRESS','MARINA EXPRESS','BART EARLY BIRD','BAYSHORE A EXPRESS',
                                'BAYSHORE B EXPRESS', 'SAN BRUNO OWL','3RD-19TH AVE OWL','INGLESIDE BUS','OWL TARAVAL',
                                'JUDAH BUS','OWL JUDAH','THIRD BUS']


muni[['route_number', 'route_long_name']] = muni['Route'].str.split(' ', n = 1, expand = True)

In [9]:
route_stops['route_long_name'] = route_stops['route_long_name'].replace(standardize_route_stops_long_name_dictionary)
muni['route_long_name'] = muni['route_long_name'].replace(muni_route_long_name_dictionary)

muni['route_number'] = muni['route_number'].replace(muni_route_number_dictionary)

muni = muni[~muni['route_long_name'].isin(muni_drop_suspended_routes_list)]
route_stops = route_stops[~route_stops['route_long_name'].isin(route_stops_drop_routes_list)]

muni = muni[['Month', 'Average Daily Boardings', 'route_number', 'route_long_name']]
route_stops = route_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'route_long_name']] 

In [10]:
## get geo dataframes for route_stops and bikeshare_station -> create a 400 meter buffer size for each bus stop -> 
# -> find which stations fall inside each buffer ->
# -> drop duplicate stations that appear in each route -> join each route-month from muni with the stations near that route -> 
# -> keep only the stations that already exist by that Month -> count unique stations per route-month -> merge back into muni panel

In [11]:
stops_gdf = gpd.GeoDataFrame(route_stops, geometry = gpd.points_from_xy(route_stops['stop_lon'], route_stops['stop_lat'], crs = 'EPSG:4326').to_crs(epsg = 3857))
stations_gdf = gpd.GeoDataFrame(bikeshare_stations, geometry = gpd.points_from_xy(bikeshare_stations['end_lng'], bikeshare_stations['end_lat']), crs = 'EPSG:4326').to_crs(epsg = 3857)

stops_buffered = stops_gdf.copy()
stops_buffered['geometry'] = stops_buffered.geometry.buffer(400)

route_stop_station = gpd.sjoin(stations_gdf, stops_buffered, predicate = 'within', how = 'inner')

route_station_pairs = route_stop_station[['route_long_name', 'end_station_id', 'first_appeared_at']].drop_duplicates() 
# THE STATIONS WITHIN 400 METERS OF AT LEAST ONE STOP ON EACH ROUTE

In [12]:
route_month = muni[['route_long_name', 'Month']].drop_duplicates()

route_month_station = route_month.merge(route_station_pairs, on = 'route_long_name', how = 'left')

mask_active = route_month_station['first_appeared_at'] <= route_month_station['Month']
route_month_station = route_month_station[mask_active]

route_month_station_counts = route_month_station.groupby(['route_long_name', 'Month'])['end_station_id'].nunique().reset_index(name = 'unique_stations_within_400m')

muni_with_station_counts = muni.merge(route_month_station_counts, on = ['route_long_name', 'Month'], how = 'left').fillna({'unique_stations_within_400m': 0})

muni_with_station_counts['treated'] = (muni_with_station_counts['unique_stations_within_400m'] > 0).astype(int)

In [13]:
# Is there enough routes that start with 0 stations within 400 meters and later gain stations within 400 meters (switchers)?
# The coefficient estimate is found only by routes where treatment changes over time (the switchers). Routes that are always treated or never treated
# do not contribute to the coefficient estimate

df = muni_with_station_counts.copy()

route_treatment_summary = (
    df.groupby('route_number')['treated'].agg(['min','max','mean'])
    .assign(
        group = lambda d: np.select(
            [(d['max'] == 0), 
             (d['min'] == 0) & (d['max'] == 1), 
             (d['min'] == 1)
            ], 
            ['never_treated', 'switcher', 'always_treated'], 
            default = 'other'
        )
    )
)

route_treatment_counts = route_treatment_summary['group'].value_counts()

In [14]:
# Run regression on entire sample 

panel = df.copy()
panel = panel.set_index(['route_number', 'Month'])

panel['treated'] = (panel['unique_stations_within_400m'] > 0).astype(int)
panel['log_boardings'] = np.log(panel['Average Daily Boardings'])

model = PanelOLS(dependent = panel['log_boardings'], exog = panel[['treated']], entity_effects = True, time_effects = True)
print(model.fit(cov_type = 'clustered', cluster_entity = True))

# There's an x percent decline in (log) average daily boardings for routes that go from 0 stations to at least 1 station within 400 meters, controlling for 
# entity and time fixed effects

                          PanelOLS Estimation Summary                           
Dep. Variable:          log_boardings   R-squared:                        0.0051
Estimator:                   PanelOLS   R-squared (Between):             -0.0414
No. Observations:                3134   R-squared (Within):               0.0490
Date:                Tue, Nov 18 2025   R-squared (Overall):             -0.0408
Time:                        22:50:44   Log-likelihood                   -575.62
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      15.417
Entities:                          44   P-value                           0.0001
Avg Obs:                       71.227   Distribution:                  F(1,3010)
Min Obs:                       40.000                                           
Max Obs:                       106.00   F-statistic (robust):             2.8068
                            

In [15]:
# Try a continuous treatment 

panel = df.copy()
panel = panel.set_index(['route_number', 'Month'])

panel['treated'] = (panel['unique_stations_within_400m'] > 0).astype(int)
panel['log_boardings'] = np.log(panel['Average Daily Boardings'])

model = PanelOLS(dependent = panel['log_boardings'], exog = panel[['unique_stations_within_400m']], entity_effects = True, time_effects = True)
print(model.fit(cov_type = 'clustered', cluster_entity = True))

                          PanelOLS Estimation Summary                           
Dep. Variable:          log_boardings   R-squared:                        0.0437
Estimator:                   PanelOLS   R-squared (Between):              0.0738
No. Observations:                3134   R-squared (Within):              -0.1073
Date:                Tue, Nov 18 2025   R-squared (Overall):              0.0759
Time:                        22:50:44   Log-likelihood                   -513.59
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      137.58
Entities:                          44   P-value                           0.0000
Avg Obs:                       71.227   Distribution:                  F(1,3010)
Min Obs:                       40.000                                           
Max Obs:                       106.00   F-statistic (robust):             22.764
                            

In [25]:
# I CAN'T LEARN MUCH ABOUT THE CAUSAL EFFECT FROM THIS SFMTA DATASET ALONE BECAUSE:
# ALMOST ALL THE ROUTES ARE ALWAYS TREATED,
# THERE ARE ALMOST NO ROUTES THAT SWITCH BETWEEN TREATED AND UNTREATED,
# THERE'S LITTLE TO NO VARIATION IN THE NUMBER OF UNIQUE STATIONS WITHIN 400 METERS (unique_stations_within_400m) OVER TIME
# in short, need more SFMTA data between 2010-2019

# aRCHIVE

In [ ]:
def _haversine_m(phi1, lam1, phi2, lam2):
    R = 6_371_000.0
    dphi = phi2 - phi1
    dlam = lam2 - lam1
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlam/2.0)**2
    return 2.0*R*np.arcsin(np.sqrt(a))

def build_route_station_exposure(route_stops: pd.DataFrame,
                                 stations: pd.DataFrame,
                                 *,
                                 radius_m: int,
                                 return_stop_station: bool = True,
                                 validate: bool = True):

    req_rs = {'route','stop_id','stop_lat','stop_lon'}
    req_st = {'station_id','station_lat','station_lon','open_month'}
    missing_rs = req_rs - set(route_stops.columns)
    missing_st = req_st - set(stations.columns)
    if missing_rs or missing_st:
        raise KeyError(f"Missing columns. route_stops: {sorted(missing_rs)} stations: {sorted(missing_st)}")

    rs = route_stops[list(req_rs)].dropna(subset=['route','stop_id','stop_lat','stop_lon']).copy()
    st = stations[list(req_st)].dropna(subset=['station_id','station_lat','station_lon']).copy()

    # Coerce types and radians
    rs['stop_lat'] = pd.to_numeric(rs['stop_lat'], errors='coerce')
    rs['stop_lon'] = pd.to_numeric(rs['stop_lon'], errors='coerce')
    st['station_lat'] = pd.to_numeric(st['station_lat'], errors='coerce')
    st['station_lon'] = pd.to_numeric(st['station_lon'], errors='coerce')

    rs = rs.dropna(subset=['stop_lat','stop_lon'])
    st = st.dropna(subset=['station_lat','station_lon'])

    rs['phi'] = np.radians(rs['stop_lat'].astype(float))
    rs['lam'] = np.radians(rs['stop_lon'].astype(float))
    st['phi'] = np.radians(st['station_lat'].astype(float))
    st['lam'] = np.radians(st['station_lon'].astype(float))
    st['open_month'] = pd.to_datetime(st['open_month'], errors='coerce').values.astype('datetime64[M]')

    n_route_stops = rs.groupby('route')['stop_id'].nunique().rename('n_route_stops')

    exposures = []
    stop_station_links = [] if return_stop_station else None

    # Pre-extract station arrays once
    st_phi = st['phi'].to_numpy()[None, :]          # shape (1, S)
    st_lam = st['lam'].to_numpy()[None, :]          # shape (1, S)

    for route, grp in rs.groupby('route', sort=False):
        g = grp.reset_index(drop=True)
        sphi = g['phi'].to_numpy()[:, None]         # shape (N, 1)
        slam = g['lam'].to_numpy()[:, None]

        # Distances from each stop on this route to every station -> (N, S)
        d = _haversine_m(sphi, slam, st_phi, st_lam)

        # Per-station summaries for this route
        min_dist = d.min(axis=0)                    # (S,)
        within = d <= float(radius_m)               # (N, S)
        n_within = within.sum(axis=0)               # (S,)

        mask = n_within > 0                         # keep only stations that are near at least one stop
        if mask.any():
            e = pd.DataFrame({
                'route': route,
                'station_id': st.loc[mask, 'station_id'].to_numpy(),
                'station_lat': st.loc[mask, 'station_lat'].to_numpy(),
                'station_lon': st.loc[mask, 'station_lon'].to_numpy(),
                'open_month': st.loc[mask, 'open_month'].to_numpy(),
                'min_dist_m': min_dist[mask],
                'n_stops_within_r': n_within[mask]
            })
            e['n_route_stops'] = int(n_route_stops.get(route, 0))
            e['stop_share_within_r'] = e['n_stops_within_r'] / e['n_route_stops'].where(e['n_route_stops'] != 0, np.nan)
            exposures.append(e)

            if return_stop_station:
                # Build stop–station rows only for stations passing mask
                idx_st = np.where(mask)[0]
                rows = []
                for j in idx_st:
                    stop_mask = within[:, j]
                    if not stop_mask.any():
                        continue
                    rows.append(pd.DataFrame({
                        'route': route,
                        'stop_id': g.loc[stop_mask, 'stop_id'].to_numpy(),
                        'station_id': st.iloc[j]['station_id'],
                        'dist_m': d[stop_mask, j],
                        'open_month': st.iloc[j]['open_month']
                    }))
                if rows:
                    stop_station_links.append(pd.concat(rows, ignore_index=True))

    exposure_df = pd.concat(exposures, ignore_index=True) if exposures else pd.DataFrame(
        columns=['route','station_id','station_lat','station_lon','open_month',
                 'min_dist_m','n_stops_within_r','n_route_stops','stop_share_within_r']
    )
    stop_station_df = (pd.concat(stop_station_links, ignore_index=True)
                       if return_stop_station and stop_station_links else None)

    if validate and not exposure_df.empty:
        assert (exposure_df['min_dist_m'] >= 0).all()
        assert (exposure_df['n_stops_within_r'] >= 1).all()
        assert (exposure_df['stop_share_within_r'].between(0, 1, inclusive='both') | exposure_df['stop_share_within_r'].isna()).all()

    return exposure_df, stop_station_df


def route_month_station_counts(panel_index: pd.DataFrame,
                               exposure_df: pd.DataFrame) -> pd.DataFrame:
    """
    panel_index: columns ['route','month'] with one row per route-month to fill.
    exposure_df: output from build_route_station_exposure().
    Returns ['route','month','n_stations_within_r','treated'].
    """
    idx = panel_index[['route','month']].copy()
    idx['month'] = pd.to_datetime(idx['month'], errors='coerce').values.astype('datetime64[M]')
    exp = exposure_df[['route','station_id','open_month']].dropna().copy()
    exp['open_month'] = pd.to_datetime(exp['open_month'], errors='coerce').values.astype('datetime64[M]')

    out = []
    for route, g in idx.groupby('route', sort=False):
        months = g['month'].to_numpy().astype('datetime64[M]').astype('int64')
        opens  = exp.loc[exp['route'] == route, 'open_month'].to_numpy().astype('datetime64[M]').astype('int64')
        opens.sort()
        counts = np.searchsorted(opens, months, side='right')
        tmp = g.copy()
        tmp['n_stations_within_r'] = counts
        tmp['treated'] = (counts > 0).astype('int8')
        out.append(tmp)
    return pd.concat(out, ignore_index=True)


def route_month_stop_coverage(panel_index: pd.DataFrame,
                              route_stops: pd.DataFrame,
                              stop_station_df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns ['route','month','stop_share_covered'] where the share is the fraction
    of a route's stops that are within radius of at least one open station by that month.
    """
    idx = panel_index[['route','month']].copy()
    idx['month'] = pd.to_datetime(idx['month'], errors='coerce').values.astype('datetime64[M]')

    n_route_stops = (route_stops
                     .groupby('route')['stop_id']
                     .nunique()
                     .rename('n_route_stops'))

    # First month a stop becomes covered by any station within radius
    cov = stop_station_df[['route','stop_id','open_month']].dropna().copy()
    cov['open_month'] = pd.to_datetime(cov['open_month'], errors='coerce').values.astype('datetime64[M]')
    cov_first = (cov.groupby(['route','stop_id'], as_index=False)['open_month']
                    .min()
                    .rename(columns={'open_month':'coverage_start'}))

    out = []
    for route, g in idx.groupby('route', sort=False):
        months = g['month'].to_numpy().astype('datetime64[M]').astype('int64')
        starts = (cov_first.loc[cov_first['route'] == route, 'coverage_start'].to_numpy()
                  .astype('datetime64[M]').astype('int64'))
        starts.sort()
        covered = np.searchsorted(starts, months, side='right')  # number of stops covered by month
        denom = float(n_route_stops.get(route, 0))
        tmp = g.copy()
        tmp['stop_share_covered'] = 0.0 if denom == 0 else covered / denom
        out.append(tmp)
    return pd.concat(out, ignore_index=True)

In [ ]:
bikeshare_stations['open_month'] = pd.to_datetime(bikeshare_stations['first_appeared_at']).values.astype('datetime64[M]')
bikeshare_stations.rename(columns = {'end_station_id': 'station_id', 'end_lng': 'station_lon', 'end_lat':'station_lat'}, inplace = True)

exposure200, stop_station200 = build_route_station_exposure(route_stops, bikeshare_stations, radius_m = 25, return_stop_station = True)
exposure300, stop_station300 = build_route_station_exposure(route_stops, bikeshare_stations, radius_m = 50, return_stop_station = True)
exposure400, stop_station400 = build_route_station_exposure(route_stops, bikeshare_stations, radius_m = 75, return_stop_station = True)
exposure500, stop_station500 = build_route_station_exposure(route_stops, bikeshare_stations, radius_m = 100, return_stop_station = True)

In [ ]:
treat_counts = route_month_station_counts(ridership, exposure400) # 
stop_cov     = route_month_stop_coverage(ridership, route_stops, stop_station400) #

for df in (ridership, treat_counts, stop_cov):
    df['month'] = pd.to_datetime(df['month'], errors='coerce').dt.to_period('M')
    df.dropna(subset=['month'], inplace=True)

panel = (ridership
         .merge(treat_counts, on=['route','month'], how='left')
         .merge(stop_cov,     on=['route','month'], how='left')
         .assign(stop_share_covered=lambda d: d['stop_share_covered'].fillna(0.0),
                 n_stations_within_r=lambda d: d['n_stations_within_r'].fillna(0).astype(int),
                 treated=lambda d: d['treated'].fillna(0).astype('int8')))

panel = panel[panel['Service Day of the Week'] == 'Weekday']

col = (panel['Average Daily Boardings']
         .astype(str)
         .str.replace(',', '', regex=False)
         .str.strip())
panel['Average Daily Boardings'] = pd.to_numeric(col, errors='coerce')
panel['log_boardings'] = np.log1p(panel['Average Daily Boardings'].astype(float))
panel['month'] = panel['month'].dt.to_timestamp(how = 'start')

panel = panel[['month', 'route', 'n_stations_within_r', 'stop_share_covered', 'treated','log_boardings']]
panel.rename(columns = {'month':'year_month', 'n_stations_within_r':'stations_within_radius', 'stop_share_covered':'share_of_stops_within_radius'}, inplace = True)
panel

-----

In [ ]:
from linearmodels import PanelOLS

panel = panel.set_index(['route', 'year_month']).sort_index()

twfe_model = PanelOLS(panel['log_boardings'], panel[['treated']], entity_effects = True, time_effects = True)
twfe_model.fit(cov_type = 'clustered', cluster_entity = True)

In [ ]:
panel['treated'].value_counts()

-----

In [ ]:
df = panel.copy()
df['year_month'] = pd.to_datetime(df['year_month']).dt.to_period('M')
df = df.sort_values(['route', 'year_month'])

prev = df.groupby('route')['treated'].shift(fill_value=0)
df['became_treated'] = df['treated'].eq(1) & prev.eq(0)

first_treat = (
    df.loc[df['became_treated'], ['route', 'year_month']]
      .groupby('route', as_index=False)
      .min()
      .rename(columns={'year_month': 'first_treat_month'})
)

first_treat_all = df[['route']].drop_duplicates().merge(first_treat, on='route', how='left')
first_treat.groupby('first_treat_month').count()

-----

In [ ]:
df = panel.copy()

df['year_month'] = pd.PeriodIndex(pd.to_datetime(df['year_month']).dt.to_period('M'))
df = df.sort_values(['route', 'year_month'])
df = df.set_index(['route', 'year_month'])

y = df['log_boardings']

df_reset = df.reset_index()

first_event_by_route = df_reset.loc[df_reset['treated'] > 0].groupby('route')['year_month'].min()
df_reset['event_month'] = df_reset['route'].map(first_event_by_route)

df_reset['ym_num'] = df_reset['year_month'].dt.year * 12 + df_reset['year_month'].dt.month
df_reset['em_num'] = np.where(df_reset['event_month'].isna(), np.nan, df_reset['event_month'].dt.year * 12 + df_reset['event_month'].dt.month)

df_reset['event_time'] = df_reset['ym_num'] - df_reset['em_num']
df = df_reset.set_index(['route', 'year_month']).sort_index()

k_min = -3
k_max = 5
baseline_k = 0.0
event_window = list(range(k_min, k_max + 1))

dummy_cols = []
for k in event_window:
    if k == baseline_k:
        continue
    col = f'event_k_{k:+d}'
    df[col] = (df['event_time'] == k).astype('float64')
    dummy_cols.append(col)

X = df[dummy_cols]
y = df['log_boardings']

df = df.copy().reset_index()
df['year_month'] = df['year_month'].dt.to_timestamp(how = 'start')

df = df.sort_values(['route', 'year_month']).set_index(['route', 'year_month'])

y = df['log_boardings'].astype('float64')
X = df.filter(like = 'event_k_').astype('float64')

model = PanelOLS(y, X, entity_effects = True, time_effects = True)
res = model.fit(cov_type = 'clustered', cluster_entity = True, cluster_time = True)

params = res.params.filter(like='event_k_')
ses = res.std_errors.filter(like='event_k_')
pvals = res.pvalues.filter(like='event_k_')
k_values = pd.Index(params.index).str.replace('event_k_', '', regex=False).astype(int)

event_results = pd.DataFrame({
    'k': k_values.values,
    'coef': params.values,
    'se': ses.values,
    'pvalue': pvals.values
}).sort_values('k').reset_index(drop=True)
event_results['ci_low'] = event_results['coef'] - 1.96 * event_results['se']
event_results['ci_high'] = event_results['coef'] + 1.96 * event_results['se']
event_results['percent_change'] = 100 * (np.exp(event_results['coef']) - 1)

import plotnine as p

g = (
    p.ggplot(event_results, p.aes(x = 'k', y = 'coef', 
                              ymin = 'ci_low', ymax = 'ci_high')) +\
    p.geom_hline(yintercept = 0, linetype = 'dotted') +\
    p.geom_pointrange(color = 'red', size = 0.7) +\
    p.theme_minimal() +\
    p.geom_hline(yintercept = 0, linetype = 'dashed') +\
    p.geom_vline(xintercept = 0, linetype = 'dashed') +\
    p.coord_cartesian(ylim = (-0.6, 0.6)) +\
    p.xlab('Months before and after first bikeshare installation') +\
    p.ylab('log(Average Daily Boardings)') +\
    p.labs(title='Figure 2: Event Study (300 Meters)') +\
    p.theme(figure_size=(13, 8))
)

g

In [ ]:
# rows actually used in the regression
valid_idx = y.index[~y.isna() & ~X.isna().any(axis=1)]

rows = []
for k in range(k_min, k_max + 1):
    if k == baseline_k:  # skip the omitted baseline
        continue
    col = f'event_k_{k:+d}'
    xk = X.loc[valid_idx, col]
    rows.append({
        'k': k,
        'n_routes': int(xk[xk == 1].index.get_level_values('route').nunique()),
        'n_obs': int(xk.sum())
    })

counts = pd.DataFrame(rows).sort_values('k')
# optional: attach to your coefficients table
event_results = event_results.merge(counts, on='k', how='left')
event_results

In [ ]:
# treated routes only
d = df_reset.loc[df_reset['event_month'].notna(), ['route','event_time','log_boardings']]

# count observed pre months per route in {-3,-2,-1}
pre_counts = (
    d.loc[d['event_time'].between(-3, -1) & ~d['log_boardings'].isna()]
     .groupby('route').size()
)

# include treated routes with zero pre months
pre_counts = pre_counts.reindex(d['route'].unique(), fill_value=0)

# result
n_routes_lacking = int((pre_counts < 3).sum())
routes_lacking = pre_counts[pre_counts < 3].index.tolist()
n_routes_lacking

In [ ]:
event_results

In [ ]:
EARTH_RADIUS_METERS = 6371000
def haversine_distance(bus_latitude: float, bus_longitude: float, station_latitude: float, station_longitude: float, radius: int):

    bus_latitude = np.radians(bus_latitude)
    bus_longitude = np.radians(bus_longitude)
    station_latitude = np.radians(station_latitude)
    station_longitude = np.radians(station_longitude)

    difference_latitude = station_latitude - bus_latitude
    difference_longitude = station_longitude - bus_longitude

    half_chord_squared = (np.sin(difference_latitude / 2.0)**2) + (np.cos(bus_latitude)) * (np.cos(station_latitude)) * (np.sin(difference_longitude / 2.0)**2)
    angular_distance = 2 * (np.arcsin(np.sqrt(half_chord_squared)))

    return radius * angular_distance